### Imports

In [1]:
import torch
from torch import nn
import torch.nn.functional as F
from torch import optim

from tqdm import tqdm
import matplotlib.pyplot as plt
import torchmetrics as tm
import numpy as np
import pandas as pd
import cv2
import pandas
import os
from sklearn.utils import shuffle

from pathlib import Path

from types import MethodType
from facenet_pytorch import InceptionResnetV1, MTCNN
import torchmetrics
import utils 
from utils import load, save, plot

from torch.utils.data import Dataset
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

### Arguments

In [23]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')#'cpu'
save_path = './save_model/model.pth'

train_batch_size = 128
val_batch_size = 128

### Dataset

making CSV file from dataset

In [15]:
root_dir = './dataset/0/'
id = 0
image_paths = []
labels = []

for class_label, name in enumerate(os.listdir(root_dir)):
    # print(class_label, name)
    img_path = os.path.join(root_dir, name)
    if name.endswith('.jpg'):
        image_paths.append(img_path)
        labels.append(id)

df = pd.DataFrame({
    'image_paths': image_paths,
    'labels': labels
})


file_path = './dataset/0.csv'
df.to_csv(file_path, index=False)

Merging two dataframe

In [16]:
df1 = pd.read_csv('./dataset/0.csv')
df2 = pd.read_csv('./dataset/1.csv')

# Concatenate the DataFrames
combined_df = pd.concat([df1, df2], ignore_index=True)

# Shuffle the combined DataFrame
shuffled_df = shuffle(combined_df, random_state=42)  # random_state for reproducibility

# Save the shuffled DataFrame to a new CSV file
shuffled_df.to_csv('./dataset/dataset.csv', index=False)

Custom Dataset

In [3]:
def detect_box(self, img, save_path=None):
    # Detect faces
    batch_boxes, batch_probs, batch_points = self.detect(img, landmarks=True)
    # Select faces
    if not self.keep_all:
        batch_boxes, batch_probs, batch_points = self.select_boxes(
            batch_boxes, batch_probs, batch_points, img, method=self.selection_method
        )
    # Extract faces
    faces = self.extract(img, batch_boxes, save_path)
    return batch_boxes, faces

Detection Model

In [4]:
mtcnn = MTCNN(image_size=224, keep_all=True, device=device)
mtcnn.detect_box = MethodType(detect_box, mtcnn)

Trasform

In [5]:
# Define transforms
transform = transforms.Compose([
    # transforms.ToTensor(),
    transforms.Normalize([-0.0497, -0.1586, -0.2036], [0.5, 0.5, 0.5])
])

In [20]:
class CustomDataset(Dataset):
    def __init__(self, mtcnn, csv_file_path, transform, memory=False):
        self.csv_file = pd.read_csv(csv_file_path)
        self.transform = transform
        self.mtcnn = mtcnn

        self.label_list = self.csv_file['labels']

        self.memory = memory
        if memory:
            self._save_memory()

    def __len__(self,):
        return len(self.csv_file)
    
    def __getitem__(self, index):
         
        sample = self.csv_file.iloc[index]

        image = self.imgs[index] if self.memory else self._load_image(sample['image_paths'])

        label = self.label_list[index]

        if image is not None and self.transform:
            image = self.transform(image)

        return image, label#, sample['image_paths']
    
    def _save_memory(self):
        self.imgs = []
        for path in self.csv_file['image_paths']:
            self.imgs.append(self._load_image(path))

    def _load_image(self, path):
        file_path = Path(path)
        if file_path.exists():
            img = cv2.imread(path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = self._detection(img)
        else :
            img = None
        return img

    def _detection(self, x):
        _, faces = self.mtcnn.detect_box(x)
        if faces is not None and len(faces) > 0:
            face = faces[0]
        else:
            face = None
        return face

In [ ]:
img = CustomDataset(mtcnn, './dataset/dataset.csv', transform, memory=False).__getitem__(91)[0]

print(img.shape)

img = img.permute(1, 2, 0).numpy()


# Display the image
plt.imshow(img)
plt.axis('off')  
plt.show()

Clear dataset from None

In [19]:
dataset = CustomDataset(mtcnn, './dataset/dataset.csv', transform, memory=False)
num = dataset.__len__()

for i in tqdm(range(num), desc="Clearing"):
    img, label, path = dataset.__getitem__(i)
    if img is None:
        file_path = Path(path)
        if file_path.exists():
            os.remove(path)
            print('None', label)

Clearing: 100%|██████████| 5701/5701 [06:43<00:00, 14.12it/s]


Calculate std and mean

In [85]:
def compute_mean_std():
    mean = 0.0
    std = 0.0
    num_samples = 0

    dataset = CustomDataset(mtcnn, './dataset/1.csv', transform, memory=False)
    num = dataset.__len__()

    # Use tqdm to track progress
    for i in tqdm(range(num), desc="Computing mean and std"):
        img, _ = dataset.__getitem__(i)
        img = img.unsqueeze(0)  # Add batch dimension for computation

        # Compute mean and std for the current image
        mean += img.mean(0).mean(1).mean(1)  # Mean across batch dimension, then spatial dimensions
        std += img.std(0).std(1).std(1)  # Std across batch dimension, then spatial dimensions

        num_samples += 1

    # Average the mean and std
    mean /= num_samples
    std /= num_samples

    return mean, std

# Compute mean and std
mean, std = compute_mean_std()

print(f"Mean: {mean}")
print(f"Std: {std}")


Computing mean and std: 100%|██████████| 3928/3928 [08:07<00:00,  8.07it/s]

Mean: tensor([-0.0497, -0.1586, -0.2036])
Std: tensor([nan, nan, nan])


In [24]:
dataset = CustomDataset(mtcnn, './dataset/dataset.csv', transform, memory=False)

# dataset,_ = random_split(dataset,(1000, len(dataset)-1000))

train_size = int(0.8 * len(dataset))  # 80% for training
test_size = len(dataset) - train_size  # 20% for testing

# Split the dataset
train_dataset, val_dataset = random_split(dataset, [train_size, test_size])

# data_loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [25]:
train_loader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True)

# val_dataset = datasets.ImageFolder('dataset/val', transform=transform)
val_loader = DataLoader(val_dataset, batch_size=val_batch_size, shuffle=False)

In [9]:
next(iter(train_loader))[0].shape, next(iter(train_loader))[1].shape

(torch.Size([64, 3, 224, 224]), torch.Size([64]))

In [9]:
def count_files(directory):
    # List all files in the directory
    files = os.listdir(directory)
    # Filter out only the .png files
    png_files = [file for file in files if file.endswith('.jpg')]
    # Return the count of .png files
    return len(png_files)

# Example usage
directory_path = './dataset/1/'
num_png_files = count_files(directory_path)
print(f'There are {num_png_files} PNG files in the folder.')

There are 3928 PNG files in the folder.


### Train Process

 Define the `Model` - `Optimizer` - `Loss Function` - `Metric`

In [28]:
# Load the pre-trained InceptionResnetV1 model
model = InceptionResnetV1(pretrained='vggface2', classify=True, num_classes=1).to(device)

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.0001)


In [39]:
model

InceptionResnetV1(
  (conv2d_1a): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2a): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2b): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (maxpool_3a): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2d_3b): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_4a): 

In [26]:
metric = torchmetrics.Accuracy(task='binary').to(device)


#tm.MeanAbsoluteError().to(device)
loss_fn= torch.nn.BCELoss()
torch.nn.BCEWithLogitsLoss
# loss_fn = torch.nn.CrossEntropyLoss()

torch.nn.modules.loss.BCEWithLogitsLoss

Training arguments

In [27]:
loss_train_hist = []
loss_valid_hist = []

metric_train_hist = []
metric_valid_hist = []

best_loss_valid = torch.inf
epoch_counter = 0

`Training Loop`

In [29]:
epochs = 5

for epoch in range(1, epochs+1):
    ## Train
    model, loss_train, metric_train = utils.train_one_epoch(model,
                                                            train_loader,
                                                            loss_fn,
                                                            optimizer,
                                                            metric,
                                                            epoch,
                                                            device=device)
    
    ## Validation
    loss_valid, metric_valid = utils.evaluate(model,
                                              val_loader,
                                              loss_fn,
                                              metric,
                                              device=device)

        
    loss_train_hist.append(loss_train)
    loss_valid_hist.append(loss_valid)

    metric_train_hist.append(metric_train)
    metric_valid_hist.append(metric_valid)


    print(f'Train      - Loss:{loss_train}  Metric:{metric_train}')
    print(f'Validation - Loss:{loss_valid}  Metric:{metric_valid}')
    print()

    if loss_valid < best_loss_valid:
        save(save_path, model, optimizer, loss_fn)
        best_loss_valid = loss_valid
        print('Model Saved ("o")')


    epoch_counter += 1

Epoch 1: 100%|██████████| 36/36 [05:27<00:00,  9.10s/batch, loss=0.0696, metric=0.985]


Train      - Loss:0.06964140385389328  Metric:0.9850876927375793
Validation - Loss:0.005452419631183147  Metric:1.0

Model Saved ("o")


Epoch 2: 100%|██████████| 36/36 [05:41<00:00,  9.50s/batch, loss=0.00779, metric=1]


Train      - Loss:0.007791116833686829  Metric:1.0
Validation - Loss:0.005950676277279854  Metric:1.0



Epoch 3: 100%|██████████| 36/36 [05:25<00:00,  9.04s/batch, loss=0.00511, metric=1]


Train      - Loss:0.0051117208786308765  Metric:1.0
Validation - Loss:0.00431416742503643  Metric:1.0

Model Saved ("o")


Epoch 4: 100%|██████████| 36/36 [05:23<00:00,  8.98s/batch, loss=0.00376, metric=1]


Train      - Loss:0.0037568435072898865  Metric:1.0
Validation - Loss:0.0027239995542913675  Metric:1.0

Model Saved ("o")


Epoch 5: 100%|██████████| 36/36 [05:27<00:00,  9.10s/batch, loss=0.00344, metric=1]


Train      - Loss:0.0034424138721078634  Metric:1.0
Validation - Loss:0.002667441265657544  Metric:1.0

Model Saved ("o")


### Plot

In [ ]:
plot(metric_train_hist, metric_valid_hist, "Metric")
plot(loss_train_hist, loss_valid_hist, 'Loss')